In [1]:
from data.NSF_data import NSF_dataset
import torch
import numpy as np
from vis.deprecated_vis import *

def calculate_polar_coords(pc):

    calc_depth = np.linalg.norm(pc[:,:3], axis=1)
    yaw = -np.arctan2(pc[:, 1], pc[:, 0])
    pitch = np.arcsin(pc[:, 2] / (calc_depth + 1e-8))

    return yaw, pitch, calc_depth



nsf_dataloader = NSF_dataset()
for batch_id, data in enumerate(nsf_dataloader):
    pc1, pc2, gt_flow = data
    break

from loss.flow import VisibilitySmoothnessLoss

# VIS = VisibilitySmoothness(pc1, K=2, VOF=1024, HOF=50, margin=3)
# not on gpu again ... :D
# VIS(gt_flow)

(tensor(0.0005),
 tensor([[0.0000, 0.0001, 0.0002,  ..., 0.0012, 0.0008, 0.0012]]))

In [2]:
yaw, pitch, calc_depth = calculate_polar_coords(pc1[0].detach().cpu().numpy())

HOF = 1024 #
VOF = 50 # 50 degree

def get_range_img_coords(pc, VOF, HOF, ):

    yaw, pitch, calc_depth = calculate_polar_coords(pc)

    # to rads
    fov = VOF * np.pi / 180.0

    fov_down = fov / 2

    # get projections in image coords
    proj_x = 0.5 * (yaw / np.pi + 1.0)          # in [0.0, 1.0]
    # proj_y = 1.0 - (pitch + abs(fov_down)) / fov        # in [0.0, 1.0]
    proj_y = 1.0 - (pitch + abs(fov_down)) / fov        # in [0.0, 1.0]

    # scale to image size using angular resolution
    proj_x *= HOF                              # in [0.0, W]
    proj_y *= VOF                              # in [0.0, H]

    u, v = np.floor(proj_x).astype(int), np.floor(proj_y).astype(int)

    # print(np.unique(proj_y))
    depth_image = - np.ones((HOF, VOF))
    depth_image[u, v] = calc_depth

    return u, v, depth_image

u,v, depth_image = get_range_img_coords(pc1[0].detach().cpu().numpy(), VOF, HOF)

import matplotlib.pyplot as plt
from data.PATHS import TMP_VIS_PATH
plt.imshow(depth_image.T)
plt.savefig(TMP_VIS_PATH + '/depth_image.png')


In [16]:
def KNN_coords(pc, nn_ind, VOF, HOF):
    '''
    :param pc2: point cloud to strip as Nx3
    :param nn_ind: KNN indices as NxK
    :return: Coordinates of KNN in depth image as NxKx2
    '''
    H_coords, V_coords, depth_image = get_range_img_coords(pc.detach().cpu().numpy(), VOF, HOF)

    # plt.imshow(self.depth)
    # plt.savefig('toy_samples/depth.png')

    image_indices = np.stack((V_coords, H_coords)).T
    image_indices = torch.from_numpy(image_indices).to(pc.device)

    KNN_image_indices = image_indices[nn_ind]

    return KNN_image_indices

from pytorch3d.ops.knn import knn_points
dist, nn_ind, _ = knn_points(pc1, pc1, K=2)


In [43]:
image_indices = KNN_coords(pc1[0], nn_ind[0], VOF, HOF)
# unify dimensions! keep single only for your use case
from ops.visibility2D import KNN_visibility_solver, substitute_NN_by_mask
from loss.flow import smoothness_loss

def strip_KNN_with_vis(pc, nn_ind, VOF, HOF, margin=3):
    '''
    :param pc2: point cloud to strip as Nx3
    :param nn_ind: KNN indices as NxK
    :return: visibility aware KNN indices as NxK (stripped KNNs)
    '''

    _,_, depth = get_range_img_coords(pc.detach().cpu().numpy(), VOF, HOF)

    KNN_image_indices = KNN_coords(pc, nn_ind, VOF, HOF)

    depth = torch.from_numpy(depth).to(pc.device)

    valid_KNN = KNN_visibility_solver(KNN_image_indices, depth.T, margin=margin)

    visibility_aware_KNN = substitute_NN_by_mask(nn_ind, valid_KNN)

    return visibility_aware_KNN

class VisibilitySmoothness(torch.nn.Module):

    def __init__(self, pc, VOH, HOF, margin=3, loss_norm=1):
        super(VisibilitySmoothness, self).__init__()
        self.margin = margin
        self.pc = pc
        self.VOH = VOH
        self.HOV = HOF
        self.margin = margin
        self.loss_norm = loss_norm

        self.nn_ind = strip_KNN_with_vis(self.pc[0], nn_ind[0], self.VOH, self.HOV, margin=self.margin)

    def forward(self, pred_flow):
        smooth_loss, per_point_smooth_loss = smoothness_loss(pred_flow, self.nn_ind, loss_norm=self.loss_norm)

        return smooth_loss, per_point_smooth_loss

# new_nn_ind = strip_KNN_with_vis(pc1[0], nn_ind[0], VOF, HOF, margin=2)
VisSmo = VisibilitySmoothness(pc1, VOF, HOF, margin=2)

similarity_nn = (nn_ind[0] == VisSmo.nn_ind).all(1).sum() / nn_ind.shape[1]

print(similarity_nn)

#todo table of losses and results


tensor(0.9694)


NameError: name 'depth' is not defined